## Customer Churn Prediction Model

Using the [Telco customer churn data](https://www.kaggle.com/code/mechatronixs/telco-churn-prediction-feature-engineering-eda/data) from kaggle, train a machine learning model to predict customer churn.

In [1]:
import pandas as pd

In [26]:
# get training data
train = pd.read_csv("./data/training_data.csv")
# drop customer ID: not a feature for training 
train.drop("customerID", axis=1, inplace=True)

# getting validation data
val = pd.read_csv("./data/validation_data.csv")

In [27]:
train.columns

Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',
       'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [11]:
train

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Male,0,No,No,5,Yes,No,DSL,No,No,Yes,Yes,Yes,Yes,Month-to-month,No,Bank transfer (automatic),75.15,392.65,No
1,Male,0,Yes,No,66,Yes,Yes,DSL,No,No,No,Yes,No,Yes,Month-to-month,No,Electronic check,63.85,4264.6,No
2,Male,0,Yes,Yes,42,Yes,No,DSL,No,No,Yes,Yes,Yes,Yes,One year,No,Electronic check,73.15,3088.25,No
3,Male,0,No,No,19,Yes,Yes,DSL,No,No,No,No,Yes,Yes,Month-to-month,No,Mailed check,69.60,1394.55,No
4,Male,0,No,No,59,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,Yes,Bank transfer (automatic),20.20,1192.3,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5277,Male,0,No,No,1,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,No,Bank transfer (automatic),20.20,20.2,No
5278,Male,0,Yes,No,2,Yes,No,Fiber optic,No,No,Yes,No,No,No,Month-to-month,No,Electronic check,76.40,151.8,Yes
5279,Female,0,Yes,No,58,Yes,Yes,DSL,Yes,No,No,Yes,No,Yes,One year,Yes,Electronic check,68.40,3972.25,No
5280,Female,0,No,No,1,Yes,No,Fiber optic,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,75.70,75.7,Yes


In [4]:
from sklearn.preprocessing import LabelEncoder



In [5]:
categorical_columns = ['gender', 'Partner', 'Dependents',
    'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
    'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
    'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'Churn']

In [28]:
column_mapper = {}

for column in categorical_columns:
    le = LabelEncoder()
    le.fit(train.loc[:, column])
    column_mapper[column] = le

In [35]:
def pre_process_data(df, label_encoder_dict):
    df_out = df.copy()
    df_out.replace(' ', 0, inplace=True)
    df_out.loc[:, 'TotalCharges'] = pd.to_numeric(df_out.loc[:,'TotalCharges'])
    if 'customerID' in df_out.columns:
        df_out.drop('customerID', axis=1, inplace=True)
    for column, le in label_encoder_dict.items():
        df_out.loc[:, column] = le.transform(df_out.loc[:,column])
    
    return df_out

In [36]:
train_preproces = pre_process_data(train, column_mapper)
val_preproces = pre_process_data(val, column_mapper)

In [37]:
x_train = train_preproces.drop('Churn', axis=1)
y_train = train_preproces.loc[:, 'Churn'].astype(int)

In [38]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(x_train, y_train)

LogisticRegression(max_iter=1000)

In [39]:
x_val = val_preproces.drop('Churn', axis=1)
y_val = val_preproces.loc[:, 'Churn'].astype(int)

predictions = model.predict(x_val)


In [41]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_val, predictions)
print(f'Classification accuracy: {round(accuracy, 3)}')


Classification accuracy: 0.832


In [21]:
#pd.DataFrame(model.coef_, columns=x_val.columns)

NameError: name 'model' is not defined

In [43]:
import pickle

with open("./models/churn_prediction_model.pkl", "wb") as pickler:
    pickle.dump(model, pickler)

with open("./models/churn_prediction_label_encoder.pkl", "wb") as pickler:
    pickle.dump(column_mapper, pickler)

